<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/VGG19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -q
!wandb login

     |████████████████████████████████| 1.9 MB 34.8 MB/s 
     |████████████████████████████████| 182 kB 54.9 MB/s 
     |████████████████████████████████| 162 kB 59.3 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 162 kB 40.6 MB/s 
     |████████████████████████████████| 158 kB 68.7 MB/s 
     |████████████████████████████████| 157 kB 76.8 MB/s 
     |████████████████████████████████| 157 kB 74.6 MB/s 
     |████████████████████████████████| 157 kB 78.8 MB/s 
     |████████████████████████████████| 157 kB 76.1 MB/s 
     |████████████████████████████████| 157 kB 74.3 MB/s 
     |████████████████████████████████| 157 kB 76.5 MB/s 
     |████████████████████████████████| 157 kB 70.0 MB/s 
     |████████████████████████████████| 156 kB 77.4 MB/s 
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wand

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(123)
tf.config.experimental.enable_op_determinism()

In [7]:
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
vgg19 = VGG19(weights='imagenet',include_top=False,input_shape=(224,224,3))

80134624/80134624 [==============================] - 0s 0us/step


In [8]:
PATH = '/content/drive/MyDrive/Research Internship/wear particles'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 740 files belonging to 5 classes.
Using 592 files for training.
Found 740 files belonging to 5 classes.
Using 148 files for validation.


In [9]:
class_names = train_ds.class_names
print(class_names)
num_classes = len(class_names)

['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [11]:
model_vgg19 = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  vgg19,
  layers.Flatten(),
  layers.Dense(32, activation='relu'),
  layers.Dropout(0.5), 
  layers.Dense(num_classes, activation='softmax')
])
vgg19.trainable=False
model_vgg19.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_1 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 32)                802848    
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 5)                 165       
                                                                 
Total params: 20,827,397
Trainable params: 803,013
Non

In [12]:
model_vgg19.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [13]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_learning", 
      name = "vgg19",
      notes = "without augment with dropout"
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vedmanivaidya. Use `wandb login --relogin` to force relogin


In [14]:
%%time
epochs=40
history = model_vgg19.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
19/19 [==============================] - ETA: 0s - loss: 1.4054 - accuracy: 0.5524

wandb: Adding directory to artifact (/content/wandb/run-20221018_044729-k1habtd8/files/model-best)... Done. 0.8s


19/19 [==============================] - 178s 5s/step - loss: 1.4054 - accuracy: 0.5524 - val_loss: 0.6844 - val_accuracy: 0.7027
Epoch 2/40
19/19 [==============================] - ETA: 0s - loss: 0.8199 - accuracy: 0.6537

wandb: Adding directory to artifact (/content/wandb/run-20221018_044729-k1habtd8/files/model-best)... Done. 0.8s


...

19/19 [==============================] - 13s 664ms/step - loss: 0.3978 - accuracy: 0.8328 - val_loss: 0.1909 - val_accuracy: 0.9662
Epoch 35/40
19/19 [==============================] - 8s 410ms/step - loss: 0.4233 - accuracy: 0.8142 - val_loss: 0.2198 - val_accuracy: 0.9459
Epoch 36/40
19/19 [==============================] - ETA: 0s - loss: 0.3699 - accuracy: 0.8530

wandb: Adding directory to artifact (/content/wandb/run-20221018_044729-k1habtd8/files/model-best)... Done. 0.8s


19/19 [==============================] - 12s 642ms/step - loss: 0.3699 - accuracy: 0.8530 - val_loss: 0.1889 - val_accuracy: 0.9527
Epoch 37/40
19/19 [==============================] - 8s 412ms/step - loss: 0.3970 - accuracy: 0.8361 - val_loss: 0.2082 - val_accuracy: 0.9392
Epoch 38/40
19/19 [==============================] - ETA: 0s - loss: 0.4159 - accuracy: 0.8463

wandb: Adding directory to artifact (/content/wandb/run-20221018_044729-k1habtd8/files/model-best)... Done. 0.8s


19/19 [==============================] - 13s 659ms/step - loss: 0.4159 - accuracy: 0.8463 - val_loss: 0.1739 - val_accuracy: 0.9459
Epoch 39/40
19/19 [==============================] - 8s 419ms/step - loss: 0.4674 - accuracy: 0.8074 - val_loss: 0.1958 - val_accuracy: 0.9459
Epoch 40/40
19/19 [==============================] - 8s 416ms/step - loss: 0.4212 - accuracy: 0.8142 - val_loss: 0.2166 - val_accuracy: 0.9527
CPU times: user 2min 33s, sys: 34.3 s, total: 3min 7s
Wall time: 11min 40s


In [15]:
wandb.finish()

accuracy,▁▃▄▄▄▄▄▄▅▄▅▅▅▅▆▆▆▆▆▆▆▇█▇▇▇▇▇█▇▇███▇███▇▇
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▃▃▃▃▃▂▃▃▃▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▁▁▁▁▁▁▁▁▁▁▁▂▁
val_accuracy,▁▄▃▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█▇▇▇▇▇▇▇█▇█▇▇▇█
val_loss,█▇▆▆▅▅▄▄▄▅▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▂▂▁▂▁▁▁▁▂
accuracy,0.81419
best_epoch,37
best_val_loss,0.17385
epoch,39
loss,0.42125
val_accuracy,0.9527
